In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ridgeRegression import *
from diffevol import *
import openbabel as ob
from moleculToVector import *
from thetas import *

/home/xomaiya/.conda/envs/ml/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
structs = read_dataset('test-Olesya/11_cis-pos-1.xyz')
struct_description, amber_coeffs = get_struct_description(path='test-Olesya/Initial_parameters_with_numbers_and_dihedrals_only.txt')
dataset = get_dataset(structs, struct_description)
y = np.array([struct.energy for struct in structs])

In [4]:
linear_coeffs = amber_coeffs.bonds_linear_coeffs + amber_coeffs.angles_linear_coeffs + amber_coeffs.torsions_linear_coeffs +  amber_coeffs.epsilons + amber_coeffs.qq_coeffs
thetas = amber_coeffs.bonds_zero_values + amber_coeffs.angles_zero_values + amber_coeffs.torsions_zero_phase + amber_coeffs.rmins + amber_coeffs.qs

In [5]:
H, stdH = constrH(dataset, struct_description, thetas)
y_est = H.dot(linear_coeffs)

# Энергия первой структуры по частям

In [6]:
kb = len(amber_coeffs.bonds_linear_coeffs) # длина вектора линейного коэффициента bonds
ka = len(amber_coeffs.angles_linear_coeffs) # длина вектора линейного коэффициента angles
kt = len(amber_coeffs.torsions_linear_coeffs) # линейный коэффициент torsions
kvdw = len(amber_coeffs.epsilons) # линейный коэффициент перед вдв
kq = len(amber_coeffs.qq_coeffs) # линейный коэффициент перед зарядами

yest1 = H[:, :kb].dot(amber_coeffs.bonds_linear_coeffs)
yest2 = H[:, kb:ka+kb].dot(amber_coeffs.angles_linear_coeffs)
yest3 = H[:, kb+ka:kb+ka+kt].dot(amber_coeffs.torsions_linear_coeffs)
yest4 = H[:, kb+ka+kt:kb+ka+kt+kvdw].dot(amber_coeffs.epsilons)
yest5 = H[:, kb+ka+kt+kq:kb+ka+kt+kvdw+kq].dot(amber_coeffs.qq_coeffs)

print(f' bonds: {yest1[0]},\nangles: {yest2[0]},\ntorsions: {yest3[0]},\n vdv: {yest4[0]},\n col: {yest5[0]}')

 bonds: 9.495553729354238,
angles: 20.1946032241865,
torsions: 10.581166347610619,
 vdv: -0.8802329336959719,
 col: -174.40178443909087


# Вся энергия для первой структуры

In [7]:
y_est[0]

-135.01069407163556

In [8]:
dataset.bonds_matrix[0]
dataset.angles_matrix[0]
dataset.torsions_matrix[0]

array([ 3.39674425e+00, -2.91777118e+00, -9.22829515e+00, -2.84933032e+00,
       -9.16384575e+00,  3.37519428e+00, -9.09937435e+00,  3.43567422e+00,
       -2.87484975e+00, -2.41088246e+00,  3.89871356e+00, -8.59901498e+00,
        3.63904828e+00, -8.90091971e+00, -2.54908425e+00, -8.75008456e+00,
       -2.44048854e+00,  3.91134692e+00,  3.18778655e+00, -3.17241595e+00,
        9.34313124e+00, -2.92715092e+00, -9.28735341e+00,  3.22819378e+00,
       -9.32310104e+00,  3.16626047e+00, -3.16775634e+00, -1.63804860e+00,
        4.76724859e+00, -5.77869137e+00,  6.26605813e-01,  2.54981208e+00,
       -3.61126673e+00, -2.97637196e+00,  3.25933581e+00, -9.27126928e+00,
        3.16232464e+00,  9.39803241e+00, -3.13257267e+00, -9.30914349e+00,
       -3.07343572e+00,  3.24552320e+00, -4.20159453e+00,  2.08753911e+00,
       -1.35195197e-02,  6.27561413e+00,  4.17621044e+00, -2.10103191e+00,
        4.35581848e+00, -1.91911346e+00,  1.45753758e-01, -6.12917818e+00,
       -4.06603575e+00,  

In [9]:
HH = constrHH(dataset, struct_description, thetas)

MemoryError: Unable to allocate array with shape (171, 28414, 3164) and data type float64